In [ ]:
import netCDF4 as nc
import scipy 
import os
import re
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import dask
from xmhw.xmhw import threshold, detect
from datetime import date
import cftime
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
import plotly.express as px
import plotly.graph_objects as go
import hashlib
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.ticker as ticker
import glob
import math
import seaborn as sns
from plotnine import *
from plotnine.data import mpg
from plotnine import ggplot, aes, geom_point, geom_line, facet_grid, scale_y_continuous, xlab, ylab
import matplotlib.lines as mlines

In [ ]:
from glob import glob
from scipy.spatial import cKDTree
from timeit import default_timer as timer
def find_nearest_ocean_cell(lat, lon, dataset):
    """
    Improved method to find the nearest ocean cell for a given latitude and longitude using a KDTree for spatial search.
    
    Args:
    - lat (float): Latitude of the land coordinate.
    - lon (float): Longitude of the land coordinate.
    - dataset (xarray.Dataset): The dataset containing SST data.
    
    Returns:
    - (float, float): Latitude and longitude of the nearest ocean cell with valid SST data,
                      or (None, None) if no valid ocean cell is found.
    """
    # Ensure longitude is in the same format as the dataset
    if lon < 0:
        lon += 360
    
    # Flatten the lat/lon coordinates and create a KDTree for spatial search
    lats = dataset['lat'].values
    lons = dataset['lon'].values
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    valid_mask = np.isfinite(dataset['sst'].isel(time=0, zlev=0).values.ravel())  # Check first time and level
    valid_points = np.vstack([lat_grid.ravel()[valid_mask], lon_grid.ravel()[valid_mask]]).T
    tree = cKDTree(valid_points)

    # Find the nearest valid (ocean) cell
    distance, location = tree.query([lat, lon], k=1)  # k=1 for the nearest neighbor
    nearest_lat, nearest_lon = valid_points[location]

    return nearest_lat, nearest_lon

#### Import comparable data of kelp and urchin

In [3]:
## new comparable urchin and kelp data
kelp_urchin_site = pd.read_csv("ercr_atrcrls_id.csv") #ercrct_atrc_id_v3
kelp_urchin_site["taxon"].fillna(kelp_urchin_site["RLS_category"], inplace=True)

# Drop the now redundant 'RLS_category' column
kelp_urchin_site.drop(columns=["RLS_category"], inplace=True)
kelp_urchin_site.head()

,survey_id,location,site_code,site_name,latitude,longitude,survey_year,taxon,survey_mean,survey_date,number
0,812300201,Batemans,BMP-S20,Acron Ledge,-35.7203,150.24789,2006,Ecklonia radiata,10.4,6/12/2006,0.52
1,812300202,Batemans,BMP-S20,Acron Ledge,-35.7203,150.24789,2006,Ecklonia radiata,0.0,6/12/2006,0.00
2,812300203,Batemans,BMP-S20,Acron Ledge,-35.7203,150.24789,2006,Ecklonia radiata,0.0,6/12/2006,0.10
3,812300204,Batemans,BMP-S20,Acron Ledge,-35.7203,150.24789,2006,Ecklonia radiata,0.0,6/12/2006,0.30
4,812318911,Batemans,BMP-S20,Acron Ledge,-35.7203,150.24789,2007,Ecklonia radiata,10.8,4/12/2007,0.00


In [4]:
## average into site level
annual_kelp_urchin_site = kelp_urchin_site.groupby(['location', 'survey_year', 'site_name', 'taxon', 'latitude', 'longitude']).mean(
                                                     numeric_only = True).reset_index(
                                                     level=['location', 'survey_year', 'site_name', 'taxon', 'latitude', 'longitude'])
annual_kelp_urchin_site

,location,survey_year,site_name,taxon,latitude,longitude,survey_id,survey_mean,number
0,Batemans,2005,Belowla Island South West,Ecklonia radiata,-35.55425,150.38896,812300032.5,0.0,6.930
1,Batemans,2005,Brush Island Mid North,Ecklonia radiata,-35.52617,150.41713,812300012.5,0.0,5.000
2,Batemans,2005,Brush Island Mid South,Ecklonia radiata,-35.53148,150.41545,812300022.5,12.0,1.415
3,Batemans,2005,Durras Mountain,Ecklonia radiata,-35.59893,150.34279,812300052.5,1.4,1.695
4,Batemans,2005,Kiola Gulch,Ecklonia radiata,-35.60594,150.33823,812300062.5,0.7,1.860
...,...,...,...,...,...,...,...,...,...
2536,Yorke Peninsula,2013,Haystack Island NE,Ecklonia radiata,-35.32085,136.90715,812325072.5,6.1,0.000
2537,Yorke Peninsula,2013,Haystack Island SW,Ecklonia radiata,-35.32357,136.90686,812325112.5,40.9,0.000
2538,Yorke Peninsula,2013,Seal Island SZ_East,Ecklonia radiata,-35.33763,136.92104,812325082.5,28.8,0.000
2539,Yorke Peninsula,2013,Seal Island SZ_West,Ecklonia radiata,-35.33815,136.91770,812325122.5,49.4,0.000


In [5]:
location = ['Batemans','Jervis Bay','Bicheno', 'Maria Island', 'Kent Group']
annual_kelp_urchin_site = annual_kelp_urchin_site[annual_kelp_urchin_site.location.isin(location)].reset_index(drop=True)
df_merge = annual_kelp_urchin_site
df_merge.head()


,location,survey_year,site_name,taxon,latitude,longitude,survey_id,survey_mean,number
0,Batemans,2005,Belowla Island South West,Ecklonia radiata,-35.55425,150.38896,812300032.5,0.0,6.930
1,Batemans,2005,Brush Island Mid North,Ecklonia radiata,-35.52617,150.41713,812300012.5,0.0,5.000
2,Batemans,2005,Brush Island Mid South,Ecklonia radiata,-35.53148,150.41545,812300022.5,12.0,1.415
3,Batemans,2005,Durras Mountain,Ecklonia radiata,-35.59893,150.34279,812300052.5,1.4,1.695
4,Batemans,2005,Kiola Gulch,Ecklonia radiata,-35.60594,150.33823,812300062.5,0.7,1.860


#### MHW temperature metrics calculation

In [ ]:
def summer_mhw_metric(ds, survey_year):
    """
    Compute summer marine heatwave (MHW) metrics for a given survey year.
    """
    # Define the time range for the summer season (Dec–Apr)
    start_date = f"{survey_year-1}-12-01"
    end_date = f"{survey_year}-04-30"
    
    ds_summer = ds.sel(time=slice(start_date, end_date))

    if ds_summer.sst.isnull().all():
        return pd.DataFrame()  # Return empty DataFrame if no valid SST data

    # Convert time to DataFrame
    date = np.array(ds_summer.time, dtype="datetime64[ns]")
    df = pd.DataFrame(date, columns=["date"])
    df["doy"] = df["date"].dt.dayofyear
    df["temp"] = pd.DataFrame(ds_summer.sst.values)

    # Compute climatology and threshold
    clim = threshold(ds.sst, pctile=90, climatologyPeriod=[1991, 2020])
    maxclim = clim.seas.max().values

    ## ========================= Detect MHW Events =========================
    mhw_detected = detect(ds.sst, clim.thresh, clim.seas)

    mhw_events_df = pd.DataFrame({"start_date": mhw_detected["time_start"],
                                  "end_date": mhw_detected["time_end"],
                                  "duration": mhw_detected["duration"]})

    # Filter MHWs occurring in the relevant summer period
    mhw_summer_df = mhw_events_df[
        (pd.to_datetime(mhw_events_df["start_date"]).between(start_date, end_date)) &
        (pd.to_datetime(mhw_events_df["end_date"]).between(start_date, end_date))].copy()

    # Count MHWs in the selected summer
    nmhw = len(mhw_summer_df)
    mean_dur = mhw_summer_df.duration.mean()

    ## ========================= Temperature Metrics =========================
    # Convert climatology to DataFrame and merge
    climatology_df = clim.to_dataframe().reset_index()
    merged_df = pd.merge(df, climatology_df, on="doy", how="left")

    # Calculate heatwave intensity and duration
    merged_df["dhd50"] = (merged_df.temp - maxclim).where((merged_df.temp - maxclim) > 0, 0)
    merged_df["max_inten"] = merged_df.temp - maxclim

    # Aggregate metrics for the summer period
    summer_mhw_metrix = merged_df[["temp", "dhd50", "max_inten"]].set_index(merged_df.date)
    summer_mhw_metrix.columns = ["summer_temp", "summer_dhd50", "summer_inten"]

    summer_mhw_metrix = summer_mhw_metrix.resample("AS-DEC").agg({
        "summer_temp": "mean",
        "summer_dhd50": "sum",
        "summer_inten": "mean"
    })

    # Assign survey year and MHW count
    summer_mhw_metrix["survey_year"] = survey_year
    summer_mhw_metrix["mhw_count"] = nmhw
    summer_mhw_metrix["duration"] = mean_dur

    return summer_mhw_metrix.reset_index(drop=True)

In [55]:
ds = xr.open_dataset('OBIS_kelp_sst.nc') #.sel(time = slice('1991-01-01','2023-12-31'))

df_mhw = pd.DataFrame([])

for i, row in df_merge.iterrows():
    location = row['location']
    
    lat = row['latitude']
    lon = row['longitude']
    
    # select specific location
    # print('extracting sst...')
    ds_locat = ds.sel(lat = lat, lon = lon, method='nearest').squeeze('zlev', drop=True) #.mean(dim=("lat", "lon"))
    # Skip this iteration if the data is all NaNs
    if ds_locat.sst.isnull().all():     
        nearest_lat, nearest_lon = find_nearest_ocean_cell(lat, lon, ds)
        
        if nearest_lat is not None and nearest_lon is not None:
            ds_locat = ds.sel(lat = nearest_lat, lon = nearest_lon).squeeze('zlev', drop=True) #.mean(dim=("lat", "lon"))
        else:
            print(f"No valid ocean cell found for: {location}")
            continue  # Skip this location
    
    # print('calculating mhws...')
    # Compute MHW metrics only for the specific survey year
    df_mhw_row = summer_mhw_metric(ds_locat, row["survey_year"])
    
    df_mhw = pd.concat([df_mhw, df_mhw_row])
df_mhw

,summer_temp,summer_dhd50,summer_inten,survey_year,nmhw,duration
0,21.587351,22.491304,-0.798670,2005,0,NaN
0,21.587351,22.491304,-0.798670,2005,0,NaN
0,21.587351,22.491304,-0.798670,2005,0,NaN
0,21.587351,22.491304,-0.798670,2005,0,NaN
0,21.587351,22.491304,-0.798670,2005,0,NaN
...,...,...,...,...,...,...
0,16.371721,52.575237,-0.239588,2023,3,6.333333
0,16.371721,52.575237,-0.239588,2023,3,6.333333
0,16.371721,52.575237,-0.239588,2023,3,6.333333
0,16.371721,52.575237,-0.239588,2023,3,6.333333


In [61]:
df_concat_mhw = pd.concat([df_merge, df_mhw[['summer_temp', 'summer_dhd50', 'summer_inten','nmhw', 'duration']].reset_index(drop=True)], axis = 1)#.dropna(subset=["survey_change"])
df_concat_mhw[['duration']] = df_concat_mhw[['duration']].fillna(0)

### Calculate MHW intensity levels

In [63]:
df_concat_mhw = df_concat_mhw[df_concat_mhw.longitude > 140].reset_index(drop = True)
df_kelpurchin_site = df_concat_mhw.groupby(['location', 'site_name', 'latitude', 'longitude']).mean(
                                                     numeric_only = True).reset_index(
                                                     level=['location', 'site_name', 'latitude', 'longitude'])
df_kelpurchin_site = df_kelpurchin_site[['location', 'site_name', 'latitude', 'longitude']]
df_kelpurchin_site

,location,site_name,latitude,longitude
0,Batemans,Acron Ledge,-35.72030,150.24789
1,Batemans,Barlings Beach N,-35.78062,150.23433
2,Batemans,Barren Bommie,-36.13559,150.12916
3,Batemans,Belowla Island South West,-35.55425,150.38896
4,Batemans,Bingi Bingi Point,-36.01291,150.16603
...,...,...,...,...
98,Jervis Bay,Scottish Rocks,-35.13488,150.74251
99,Jervis Bay,Tapalla Point (Husky Reef),-35.03550,150.67966
100,Jervis Bay,The Docks 1 The Gulch,-35.08142,150.79732
101,Jervis Bay,The Docks 2 Inner Tubes,-35.07993,150.79300


In [64]:
ds = xr.open_dataset('OBIS_kelp_sst.nc') #.sel(time = slice('1991-01-01','2021-12-31'))

df_climatmax = pd.DataFrame([])

for i, row in df_kelpurchin_site.iterrows():
    location = row['location']
    lat = row['latitude']
    lon = row['longitude']
    
    # select specific location
    ds_locat = ds.sel(lat = lat, lon = lon, method='nearest').squeeze('zlev', drop=True) #.mean(dim=("lat", "lon"))
    # Skip this iteration if the data is all NaNs
    if ds_locat.sst.isnull().all():     
        nearest_lat, nearest_lon = find_nearest_ocean_cell(lat, lon, ds)
        
        if nearest_lat is not None and nearest_lon is not None:
            ds_locat = ds.sel(lat = nearest_lat, lon = nearest_lon).squeeze('zlev', drop=True) #.mean(dim=("lat", "lon"))
        else:
            print(f"No valid ocean cell found for: {location}")
            continue  # Skip this location
    
    # print('calculating mhws...')
    clim = threshold(ds_locat.sst, pctile = 90, climatologyPeriod=[1991, 2020]) # threshold and climatology
    maxclim = float(clim.seas.max().values)  # Convert to float
    maxthresh = float(clim.thresh.max().values)
    thresh90 = clim.thresh.where((clim.doy < 122) | (clim.doy > 335), drop=True).mean().values
    seasonal50 = clim.seas.where((clim.doy < 122) | (clim.doy > 335), drop=True).mean().values
    # print(seasonal50)
    df_maxthresh = pd.DataFrame({'maxclim': [maxclim], 'maxthresh': [maxthresh], 'meanclim' : [seasonal50], 'meanthresh90' : [thresh90]})
    
    df_climatmax = pd.concat([df_climatmax, df_maxthresh])  
df_climatmax

0.3.0


,maxclim,maxthresh,meanclim,meanthresh90
0,22.386021,23.581365,21.581018,22.89991216037526
0,22.445671,23.643247,21.633276,22.959975223657306
0,22.224281,23.370053,21.37685,22.682769510080806
0,22.386021,23.581365,21.581018,22.89991216037526
0,22.224281,23.370053,21.37685,22.682769510080806
...,...,...,...,...
0,23.052973,24.518795,22.175598,23.573373140746348
0,23.052973,24.518795,22.175598,23.573373140746348
0,23.052973,24.518795,22.175598,23.573373140746348
0,23.052973,24.518795,22.175598,23.573373140746348


In [67]:
df_concat_climat = pd.concat([df_kelpurchin_site, df_climatmax.reset_index(drop=True)], axis = 1)#.dropna(subset=["survey_change"])
df_concat_climat['inten_level1'] = df_concat_climat.maxthresh - df_concat_climat.maxclim
df_concat_climat['inten_level2'] = 2*(df_concat_climat.maxthresh- df_concat_climat.maxclim)
df_concat_climat['inten_level3'] = 3*(df_concat_climat.maxthresh- df_concat_climat.maxclim)
df_concat_climat

,location,site_name,latitude,longitude,maxclim,maxthresh,meanclim,meanthresh90,inten_level1,inten_level2,inten_level3
0,Batemans,Acron Ledge,-35.72030,150.24789,22.386021,23.581365,21.581018,22.89991216037526,1.195344,2.390689,3.586033
1,Batemans,Barlings Beach N,-35.78062,150.23433,22.445671,23.643247,21.633276,22.959975223657306,1.197576,2.395151,3.592727
2,Batemans,Barren Bommie,-36.13559,150.12916,22.224281,23.370053,21.37685,22.682769510080806,1.145772,2.291543,3.437315
3,Batemans,Belowla Island South West,-35.55425,150.38896,22.386021,23.581365,21.581018,22.89991216037526,1.195344,2.390689,3.586033
4,Batemans,Bingi Bingi Point,-36.01291,150.16603,22.224281,23.370053,21.37685,22.682769510080806,1.145772,2.291543,3.437315
...,...,...,...,...,...,...,...,...,...,...,...
98,Jervis Bay,Scottish Rocks,-35.13488,150.74251,23.052973,24.518795,22.175598,23.573373140746348,1.465822,2.931644,4.397467
99,Jervis Bay,Tapalla Point (Husky Reef),-35.03550,150.67966,23.052973,24.518795,22.175598,23.573373140746348,1.465822,2.931644,4.397467
100,Jervis Bay,The Docks 1 The Gulch,-35.08142,150.79732,23.052973,24.518795,22.175598,23.573373140746348,1.465822,2.931644,4.397467
101,Jervis Bay,The Docks 2 Inner Tubes,-35.07993,150.79300,23.052973,24.518795,22.175598,23.573373140746348,1.465822,2.931644,4.397467


In [ ]:
df_concat_mhw_level = df_concat_mhw.merge(df_concat_climat, how='inner', on = ['location', 'site_name', 'longitude','latitude'])

In [71]:
df_concat_mhw_level.to_csv('all_ukmhw_with_level.csv', index = False)